In [ ]:
import polars as pl
import numpy as np
import networkx as nx
import random
from math import cos, sin, pi ,sqrt
import rtsvg
rt  = rtsvg.RACETrack()

circles = [(25,25,20),(80,80,30),(20,100,10),(25,60,5),(60,30,5),(48,48,6),(40,90,6)]

svg_header = ['<svg x="0" y="0" viewBox="0 0 128 128" width="512" height="512">']
svg_header.append(f'<rect x="0" y="0" width="128" height="128" fill="#ffffff" />')
svg_circles = []
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    svg_circles.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="{_color_}" fill="none" />')
    svg_circles.append(f'<circle cx="{cx}" cy="{cy}" r="0.2" stroke="#000000" fill="#000000" />')
    svg_circles.append(rt.svgText(str(i), cx, cy-1, 4, anchor='middle'))
svg_footer = ['</svg>']
rt.tile([''.join(svg_header) + ''.join(svg_circles) + ''.join(svg_footer)])

In [2]:
#
# Point Version
#
svg = ['<svg x="0" y="0" viewBox="0 0 128 128" width="512" height="512">']
svg.append(f'<rect x="0" y="0" width="128" height="128" fill="#ffffff" />')
cells   = rt.isedgarVoronoi(circles, [(0,0), (0,128), (128,128), (128,0)], use_circle_radius=False)
#svg.append('<rect x="23" y="60" width="10" height="10" fill="none" stroke="#000000" stroke-width="0.5" />')
def distanceToCircle(_xy_, i):
    return sqrt((circles[i][0]-_xy_[0])**2 + (circles[i][1]-_xy_[1])**2)
for box in [(0,128,0,128)]:
    x_l, x_r, y_t, y_b = box
    for x in range(x_l, x_r):
        for y in range(y_t, y_b):
            closest_circle_i = 0
            closest_d        = distanceToCircle((x,y), 0)
            for i in range(1, len(circles)):
                d = distanceToCircle((x,y), i)
                if d < closest_d:
                    closest_d        = d
                    closest_circle_i = i
            #svg.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{rt.co_mgr.getColor(closest_circle_i)}" stroke="none" stroke-width="0.5" />')
for cell in cells:
    d = f'M {cell[0][0]} {cell[0][1]} '
    for pt in cell[1:]: d += f'L {pt[0]} {pt[1]} '
    d += 'Z'
    _color_ = '#000000' # rt.co_mgr.getColor(str(cell))
    svg.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="0.2" />')
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    svg.append(f'<circle cx="{cx}" cy="{cy}" r="{r}" stroke="#000000" fill="none" />')
    svg.append(f'<circle cx="{cx}" cy="{cy}" r="1" stroke="#000000" fill="#000000" />')
svg.append('</svg>')
#rt.tile([''.join(svg)]) # point version

In [ ]:
#
# Circle Version
#
cells   = rt.isedgarVoronoi(circles, [(0,0), (0,128), (128,128), (128,0)], use_circle_radius=True)
def distanceToCircle(_xy_, i):
    return sqrt((circles[i][0]-_xy_[0])**2 + (circles[i][1]-_xy_[1])**2) - circles[i][2]
for box in [(0,128,0,128)]:
    x_l, x_r, y_t, y_b = box
    for x in range(x_l, x_r):
        for y in range(y_t, y_b):
            closest_circle_i = 0
            closest_d        = distanceToCircle((x,y), 0)
            for i in range(1, len(circles)):
                d = distanceToCircle((x,y), i)
                if d < closest_d:
                    closest_d        = d
                    closest_circle_i = i
            #svg.append(f'<rect x="{x}" y="{y}" width="1" height="1" fill="{rt.co_mgr.getColor(closest_circle_i)}" stroke="none" stroke-width="0.5" />')
svg_voronoi = []
for i in range(len(circles)):
    cx,cy,r = circles[i]
    _color_ = rt.co_mgr.getColor(i)
    cell = cells[i]
    d = f'M {cell[0][0]} {cell[0][1]} '
    for pt in cell[1:]: d += f'L {pt[0]} {pt[1]} '
    d += 'Z'
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="{_color_}" stroke-width="1" />')
    svg_voronoi.append(f'<path d="{d}" fill="none" stroke="#000000" stroke-width="0.1" />')
rt.tile([''.join(svg_header) + ''.join(svg_circles) + ''.join(svg_voronoi) + ''.join(svg_footer)])

In [ ]:
poly_connects = {'__fm__':[], '__to__':[]}
for i in range(len(cells)):
    cell_i = cells[i]
    for j in range(len(cells)):
        if j == i: continue
        cell_j = cells[j]
        for ic in range(len(cell_i)):
            edge_i = (cell_i[ic],cell_i[(ic+1)%len(cell_i)])
            for jc in range(len(cell_j)):
                edge_j = (cell_j[jc],cell_j[(jc+1)%len(cell_j)])
                if rt.segmentsOverlap(edge_i, edge_j, eps=0.1): ### !!! EPS HAD TO BE MODIFIED FOR THIS SCALE
                    poly_connects['__fm__'].append(i), poly_connects['__to__'].append(j)
df_poly_connects = pl.DataFrame(poly_connects)
g_poly_connects  = rt.createNetworkXGraph(df_poly_connects, [('__fm__','__to__')])
for _cycle_ in nx.simple_cycles(g_poly_connects, 4):
    if len(_cycle_) == 4:
        print(_cycle_) # [5, 1, 6, 3] is the one w/ the problem

In [5]:
#pos = {}
#for i in range(len(circles)): pos[i] = (circles[i][0], circles[i][1])
#rt.link(df_poly_connects, [('__fm__','__to__')], pos, draw_labels=True)